# Lab 3: Soil Moisture and Hydrology

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MichalBrezny/LSPD-lab/blob/main/labs/Lab03_Soil_Moisture_Hydrology.ipynb)

## Objectives
- Understand soil water content and water potential
- Model infiltration and drainage processes
- Analyze soil moisture dynamics over time
- Explore the relationship between soil moisture and evapotranspiration

## Background
Soil moisture is a critical variable in land surface processes. It controls the partitioning of available energy into sensible and latent heat fluxes, influences surface temperature, and determines water availability for plants.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Soil parameters
POROSITY = 0.45  # m³/m³
FIELD_CAPACITY = 0.35  # m³/m³
WILTING_POINT = 0.15  # m³/m³

print("Libraries imported successfully!")

## Part 1: Soil Water Content and Water Potential

Soil moisture can be expressed in different ways:
- **Volumetric water content** (θ): volume of water per volume of soil (m³/m³)
- **Soil water potential** (ψ): energy status of water in soil (often in kPa or m)

The relationship between θ and ψ is described by the **soil water retention curve** (van Genuchten model):

$\theta = \theta_r + \frac{\theta_s - \theta_r}{[1 + (\alpha|\psi|)^n]^m}$

In [ ]:
def van_genuchten(psi, theta_r=0.05, theta_s=0.45, alpha=0.02, n=1.5):
    """
    Calculate soil water content from water potential using van Genuchten equation.
    
    Parameters:
    - psi: soil water potential (absolute value) in kPa
    - theta_r: residual water content
    - theta_s: saturated water content (porosity)
    - alpha, n: van Genuchten parameters
    
    Returns:
    - theta: volumetric water content
    """
    m = 1 - 1/n
    theta = theta_r + (theta_s - theta_r) / (1 + (alpha * np.abs(psi))**n)**m
    return theta

# Create retention curves for different soil types
psi_range = np.logspace(-1, 4, 100)  # 0.1 to 10000 kPa

soil_types = {
    'Sand': {'theta_r': 0.045, 'theta_s': 0.43, 'alpha': 0.145, 'n': 2.68},
    'Loam': {'theta_r': 0.078, 'theta_s': 0.43, 'alpha': 0.036, 'n': 1.56},
    'Clay': {'theta_r': 0.068, 'theta_s': 0.38, 'alpha': 0.008, 'n': 1.09}
}

retention_curves = {}
for soil_name, params in soil_types.items():
    theta = van_genuchten(psi_range, **params)
    retention_curves[soil_name] = theta

print("Soil water retention curves calculated!")

In [ ]:
plt.figure(figsize=(12, 7))

for soil_name, theta in retention_curves.items():
    plt.semilogx(psi_range, theta, linewidth=2, label=soil_name)

# Mark important points
plt.axhline(y=FIELD_CAPACITY, color='blue', linestyle='--', alpha=0.5, label='Field Capacity')
plt.axhline(y=WILTING_POINT, color='red', linestyle='--', alpha=0.5, label='Wilting Point')

plt.xlabel('Soil Water Potential |ψ| (kPa)', fontsize=12)
plt.ylabel('Volumetric Water Content θ (m³/m³)', fontsize=12)
plt.title('Soil Water Retention Curves', fontsize=14, fontweight='bold')
plt.legend(loc='upper right', fontsize=11)
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()

## Part 2: Infiltration and Drainage

Infiltration can be modeled using the **Green-Ampt equation**:

$f = K_s \left(1 + \frac{\psi_f \Delta\theta}{F}\right)$

where:
- $f$ = infiltration rate (mm/hr)
- $K_s$ = saturated hydraulic conductivity
- $\psi_f$ = wetting front suction
- $\Delta\theta$ = change in water content
- $F$ = cumulative infiltration

In [ ]:
def green_ampt_infiltration(t, Ks=10, psi_f=100, theta_i=0.2, theta_s=0.45, rainfall_rate=20):
    """
    Simulate infiltration using Green-Ampt model.
    
    Parameters:
    - t: time array (hours)
    - Ks: saturated hydraulic conductivity (mm/hr)
    - psi_f: wetting front suction (mm)
    - theta_i: initial water content
    - theta_s: saturated water content
    - rainfall_rate: rainfall intensity (mm/hr)
    
    Returns:
    - F: cumulative infiltration (mm)
    - f: infiltration rate (mm/hr)
    """
    dt = t[1] - t[0] if len(t) > 1 else 0.01
    F = np.zeros_like(t)
    f = np.zeros_like(t)
    
    delta_theta = theta_s - theta_i
    
    for i in range(1, len(t)):
        if F[i-1] < 0.001:
            F[i-1] = 0.001  # Avoid division by zero
        
        # Infiltration capacity
        f_capacity = Ks * (1 + (psi_f * delta_theta) / F[i-1])
        
        # Actual infiltration (limited by rainfall or capacity)
        f[i] = min(rainfall_rate, f_capacity)
        
        # Update cumulative infiltration
        F[i] = F[i-1] + f[i] * dt
    
    return F, f

# Simulate infiltration event
time_hours = np.linspace(0, 6, 200)

# Different rainfall intensities
rainfall_rates = [10, 20, 40]
infiltration_results = {}

for rain_rate in rainfall_rates:
    F, f = green_ampt_infiltration(time_hours, rainfall_rate=rain_rate)
    runoff = np.maximum(0, rain_rate - f)
    infiltration_results[f'{rain_rate} mm/hr'] = {
        'cumulative': F,
        'rate': f,
        'runoff': runoff
    }

print("Infiltration simulations complete!")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Cumulative infiltration
ax = axes[0]
for label, data in infiltration_results.items():
    ax.plot(time_hours, data['cumulative'], linewidth=2, label=f'Rain: {label}')
ax.set_xlabel('Time (hours)', fontsize=11)
ax.set_ylabel('Cumulative Infiltration (mm)', fontsize=11)
ax.set_title('Cumulative Infiltration', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Infiltration rate
ax = axes[1]
for label, data in infiltration_results.items():
    ax.plot(time_hours, data['rate'], linewidth=2, label=f'Rain: {label}')
ax.set_xlabel('Time (hours)', fontsize=11)
ax.set_ylabel('Infiltration Rate (mm/hr)', fontsize=11)
ax.set_title('Infiltration Rate', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Runoff
ax = axes[2]
for label, data in infiltration_results.items():
    ax.plot(time_hours, data['runoff'], linewidth=2, label=f'Rain: {label}')
ax.set_xlabel('Time (hours)', fontsize=11)
ax.set_ylabel('Runoff Rate (mm/hr)', fontsize=11)
ax.set_title('Surface Runoff', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Soil Moisture Dynamics

Soil moisture changes over time due to:
- Precipitation (input)
- Evapotranspiration (output)
- Drainage (output)
- Runoff (output when soil is saturated)

The soil moisture balance can be written as:

$\frac{d\theta}{dt} = P - ET - D - R$

In [ ]:
def soil_moisture_dynamics(theta, t, precip, ET_potential, theta_fc, theta_wp, theta_s, drainage_coef=0.1):
    """
    Model soil moisture dynamics.
    
    Parameters:
    - theta: current soil moisture
    - t: time
    - precip: precipitation rate (mm/day)
    - ET_potential: potential ET (mm/day)
    - theta_fc: field capacity
    - theta_wp: wilting point
    - theta_s: saturation
    - drainage_coef: drainage coefficient
    """
    # Water stress factor for ET
    if theta <= theta_wp:
        stress_factor = 0
    elif theta >= theta_fc:
        stress_factor = 1
    else:
        stress_factor = (theta - theta_wp) / (theta_fc - theta_wp)
    
    # Actual ET
    ET_actual = ET_potential * stress_factor
    
    # Drainage (occurs when above field capacity)
    if theta > theta_fc:
        drainage = drainage_coef * (theta - theta_fc)
    else:
        drainage = 0
    
    # Runoff (occurs when above saturation)
    if theta >= theta_s:
        runoff = precip  # All precipitation becomes runoff
        infiltration = 0
    else:
        runoff = 0
        infiltration = precip
    
    # Rate of change (convert from mm/day to m³/m³ per day, assuming 1m soil depth)
    d_theta_dt = (infiltration - ET_actual - drainage) / 1000
    
    return d_theta_dt

# Create synthetic weather data for 60 days
days = np.linspace(0, 60, 240)

# Precipitation events (mm/day)
np.random.seed(42)
precip_events = np.random.exponential(scale=3, size=len(days))
precip_events[np.random.rand(len(days)) > 0.2] = 0  # 20% chance of rain

# Potential ET (sinusoidal pattern)
ET_pot = 4 + 2 * np.sin(2 * np.pi * days / 60)

# Solve soil moisture dynamics
theta_initial = 0.25
theta_solution = np.zeros_like(days)
theta_solution[0] = theta_initial

for i in range(1, len(days)):
    dt = days[i] - days[i-1]
    d_theta = soil_moisture_dynamics(
        theta_solution[i-1], days[i-1], 
        precip_events[i-1], ET_pot[i-1],
        FIELD_CAPACITY, WILTING_POINT, POROSITY
    )
    theta_solution[i] = theta_solution[i-1] + d_theta * dt
    # Constrain to physical limits
    theta_solution[i] = np.clip(theta_solution[i], WILTING_POINT, POROSITY)

print("Soil moisture dynamics simulated!")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Precipitation
ax = axes[0]
ax.bar(days, precip_events, width=0.25, color='blue', alpha=0.6)
ax.set_ylabel('Precipitation (mm/day)', fontsize=11)
ax.set_title('Precipitation Events', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Potential ET
ax = axes[1]
ax.plot(days, ET_pot, 'orange', linewidth=2)
ax.fill_between(days, 0, ET_pot, alpha=0.3, color='orange')
ax.set_ylabel('Potential ET (mm/day)', fontsize=11)
ax.set_title('Potential Evapotranspiration', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Soil moisture
ax = axes[2]
ax.plot(days, theta_solution, 'green', linewidth=2, label='Soil Moisture')
ax.axhline(y=FIELD_CAPACITY, color='blue', linestyle='--', alpha=0.5, label='Field Capacity')
ax.axhline(y=WILTING_POINT, color='red', linestyle='--', alpha=0.5, label='Wilting Point')
ax.axhline(y=POROSITY, color='gray', linestyle='--', alpha=0.5, label='Saturation')
ax.fill_between(days, WILTING_POINT, FIELD_CAPACITY, alpha=0.1, color='green', label='Plant Available Water')
ax.set_xlabel('Day', fontsize=11)
ax.set_ylabel('Soil Moisture (m³/m³)', fontsize=11)
ax.set_title('Soil Moisture Dynamics', fontsize=12, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Plant Available Water
Calculate the total plant-available water (PAW) in mm for a 1-meter soil profile when soil moisture is at field capacity.

In [ ]:
# Your code here
soil_depth = 1000  # mm
PAW = (FIELD_CAPACITY - WILTING_POINT) * soil_depth

print(f"Plant Available Water at field capacity: {PAW:.0f} mm")
print(f"This represents the water available for plant uptake.")

### Exercise 2: Water Balance
Calculate the total water inputs and outputs over the 60-day period and verify the water balance.

In [ ]:
# Your code here
total_precip = np.sum(precip_events) * (days[1] - days[0])
storage_change = (theta_solution[-1] - theta_solution[0]) * 1000  # Convert to mm

# Approximate ET and drainage from the difference
outputs = total_precip - storage_change

print(f"Total precipitation: {total_precip:.1f} mm")
print(f"Change in storage: {storage_change:.1f} mm")
print(f"Total outputs (ET + drainage): {outputs:.1f} mm")
print(f"\nWater balance check: P = ΔS + Outputs")
print(f"{total_precip:.1f} ≈ {storage_change:.1f} + {outputs:.1f}")

## Discussion Questions

1. **Why do different soil types have different retention curves?**
   - Pore size distribution varies with texture
   - Clay has smaller pores, retains water at higher tensions
   - Sand has larger pores, drains more readily

2. **What happens to infiltration rate over time during a rainfall event?**
   - Decreases as soil becomes wetter
   - Wetting front advances deeper, reducing gradient
   - May lead to runoff generation

3. **How does soil moisture affect the energy balance?**
   - Controls partitioning between H and LE
   - Wet soils favor latent heat flux (evaporation)
   - Dry soils favor sensible heat flux

4. **Why is field capacity an important threshold?**
   - Represents water held against gravity
   - Above FC, water drains relatively quickly
   - Defines upper limit of plant-available water

## Summary

In this lab, you have:
- Explored soil water retention characteristics
- Modeled infiltration using the Green-Ampt equation
- Simulated soil moisture dynamics over time
- Analyzed the water balance components

## Next Steps
In Lab 4, we will examine vegetation processes and evapotranspiration in detail.